In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn import metrics
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from six.moves import cPickle as pickle
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

Using TensorFlow backend.


In [2]:
data=pd.read_csv('train.csv')
test=pd.read_csv("test.csv")

In [3]:
df43 = pd.DataFrame(data, columns = ['genre_id', 'is_listened'])
df45 = pd.DataFrame(data, columns = ['media_id', 'is_listened'])
df46=  pd.DataFrame(data, columns = ['album_id', 'is_listened'])
df47=  pd.DataFrame(data, columns = ['artist_id', 'is_listened'])
df51=  pd.DataFrame(data, columns = ['user_id', 'is_listened'])
df53=  pd.DataFrame(data, columns = ['release_date', 'is_listened'])

In [4]:
#TO create fractional features. First groupby ids and then calculate mean of is listened for every level of id
#and then map it on data using dictionary creation
count1=df43.groupby(['genre_id'],as_index=False).agg({'is_listened':'mean'})
df44=pd.Series(count1.is_listened.values,index=count1.genre_id).to_dict()
data["B"] = data["genre_id"].map(df44)
test["B"] = test["genre_id"].map(df44)


count2=df45.groupby(['media_id'],as_index=False).agg({'is_listened':'mean'})
df48=pd.Series(count2.is_listened.values,index=count2.media_id).to_dict()
data["C"] = data["media_id"].map(df48)
test["C"] = test["media_id"].map(df48)

count3=df46.groupby(['album_id'],as_index=False).agg({'is_listened':'mean'})
df49=pd.Series(count3.is_listened.values,index=count3.album_id).to_dict()
data["D"] = data["album_id"].map(df49)
test["D"] = test["album_id"].map(df49)
data.head()
count4=df47.groupby(['artist_id'],as_index=False).agg({'is_listened':'mean'})
df50=pd.Series(count4.is_listened.values,index=count4.artist_id).to_dict()
data["E"] = data["artist_id"].map(df50)
test["E"] = test["artist_id"].map(df50)

count5=df51.groupby(['user_id'],as_index=False).agg({'is_listened':'mean'})
df52=pd.Series(count5.is_listened.values,index=count5.user_id).to_dict()
data["F"] = data["user_id"].map(df52)
test["F"] = test["user_id"].map(df52)

count6=df53.groupby(['release_date'],as_index=False).agg({'is_listened':'mean'})
df54=pd.Series(count6.is_listened.values,index=count6.release_date).to_dict()
data["G"] = data["release_date"].map(df54)
test["G"] = test["release_date"].map(df54)

In [5]:
del count1
del count2
del count3
del count4
del count5
del count6
del df43
del df44
del df45
del df46
del df47
del df48
del df49
del df50
del df51
del df52

In [6]:
start_time = datetime.datetime.now()
#pivot table for clustering of similar users. Till now only used genre id for cluster
matrix = data.pivot_table(index=['user_id'], columns=['genre_id'], values='is_listened')
cluster = KMeans(n_clusters=50)
matrix=matrix.fillna(0)
x_cols = matrix.columns[1:]
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usergencluster"] = data["user_id"].map(mat2)
test["usergencluster"]=test["user_id"].map(mat2)
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()

data['album_id_1']=data['album_id']
test['album_id_1']=test['album_id']
mmo3=pd.DataFrame(data['album_id_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.album_id_1.values)],axis=1)
mmo31=list(mmo3.iloc[1500:,0])
data['album_id_1'].loc[data['album_id_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['album_id_1'], values='is_listened')
cluster = KMeans(n_clusters=50)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["useralbcluster"] = data["user_id"].map(mat2)
test["useralbcluster"]=test["user_id"].map(mat2)
del matrix
del matrix1
del mat2
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()

data['media_id_1']=data['media_id']
test['media_id_1']=test['media_id']
mmo3=pd.DataFrame(data['media_id_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.media_id_1.values)],axis=1)
mmo31=list(mmo3.iloc[3000:,0])
data['media_id_1'].loc[data['media_id_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['media_id_1'], values='is_listened')
cluster = KMeans(n_clusters=50)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usermedcluster"] = data["user_id"].map(mat2)
test["usermedcluster"]=test["user_id"].map(mat2)
del matrix
del matrix1
del mat2
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()

data['artist_id_1']=data['artist_id']
test['artist_id_1']=test['artist_id']
mmo3=pd.DataFrame(data['artist_id_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.artist_id_1.values)],axis=1)
mmo31=list(mmo3.iloc[1700:,0])
data['artist_id_1'].loc[data['artist_id_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['artist_id_1'], values='is_listened')
cluster = KMeans(n_clusters=50)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userartcluster"] = data["user_id"].map(mat2)
test["userartcluster"]=test["user_id"].map(mat2)
del matrix
del matrix1
del mat2
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()

data['release_date_1']=data['release_date']
test['release_date_1']=test['release_date']
mmo3=pd.DataFrame(data['release_date_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.release_date_1.values)],axis=1)
mmo31=list(mmo3.iloc[1000:,0])
data['release_date_1'].loc[data['release_date_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['release_date_1'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userdatecluster"] = data["user_id"].map(mat2)
test["userdatecluster"]=test["user_id"].map(mat2)
del matrix
del matrix1
del mat2
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()

0:05:53.485949


/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0:02:40.903218
0:04:22.317093
0:02:51.561791
0:00:53.092824


In [7]:
data.to_pickle("fulldata.pickle")
test.to_pickle("fulltest.pickle")

In [2]:
data = pd.read_pickle("fulldata.pickle")
test = pd.read_pickle("fulltest.pickle")

In [3]:
matrix = data.pivot_table(index=['user_id'], columns=['platform_family'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
x_cols = matrix.columns[1:]
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userplafamcluster"] = data["user_id"].map(mat2)
test["userplafamcluster"]=test["user_id"].map(mat2)


matrix = data.pivot_table(index=['user_id'], columns=['listen_type'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
x_cols = matrix.columns[1:]
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userlistypecluster"] = data["user_id"].map(mat2)
test["userlistypecluster"]=test["user_id"].map(mat2)


matrix = data.pivot_table(index=['user_id'], columns=['platform_name'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
x_cols = matrix.columns[1:]
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userplnamecluster"] = data["user_id"].map(mat2)
test["userplnamecluster"]=test["user_id"].map(mat2)


matrix = data.pivot_table(index=['user_id'], columns=['context_type'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
x_cols = matrix.columns[1:]
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usercontypecluster"] = data["user_id"].map(mat2)
test["usercontypecluster"]=test["user_id"].map(mat2)

matrix = data.pivot_table(index=['user_id'], columns=['user_age'], values='is_listened')
cluster = KMeans(n_clusters=5)
matrix=matrix.fillna(0)
x_cols = matrix.columns[1:]
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["useruagecluster"] = data["user_id"].map(mat2)
test["useruagecluster"] = test["user_id"].map(mat2)

del matrix
del matrix1
del mat2

In [4]:
mmo3=pd.DataFrame(data['album_id'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.album_id.values)],axis=1)
mmo31=list(mmo3.iloc[50:,0])
data['album_id'].loc[data['album_id'].isin(mmo31)]=-1
test['album_id'].loc[test['album_id'].isin(mmo31)]=-1

mmo4=pd.DataFrame(data['artist_id'].value_counts())
mmo4=pd.concat([pd.DataFrame(mmo4.index.values),pd.DataFrame(mmo4.artist_id.values)],axis=1)
mmo41=list(mmo4.iloc[50:,0])
data['artist_id'].loc[data['artist_id'].isin(mmo41)]=-1
test['artist_id'].loc[test['artist_id'].isin(mmo41)]=-1

      
mmo5=pd.DataFrame(data['genre_id'].value_counts())
mmo5=pd.concat([pd.DataFrame(mmo5.index.values),pd.DataFrame(mmo5.genre_id.values)],axis=1)
mmo51=list(mmo5.iloc[50:,0])
data['genre_id'].loc[data['genre_id'].isin(mmo51)]=-1
test['genre_id'].loc[test['genre_id'].isin(mmo51)]=-1
      
mmo6=pd.DataFrame(data['context_type'].value_counts())
mmo6=pd.concat([pd.DataFrame(mmo6.index.values),pd.DataFrame(mmo6.context_type.values)],axis=1)
mmo61=list(mmo6.iloc[15:,0])
data['context_type'].loc[data['context_type'].isin(mmo61)]=-1
test['context_type'].loc[test['context_type'].isin(mmo61)]=-1

data['release_date1']=data.release_date
test['release_date1']=test.release_date
mmo7=pd.DataFrame(data['release_date'].value_counts())
mmo98=pd.DataFrame(test['release_date'].value_counts())
unmatched = set(list(mmo98.index.values))-set(list(mmo7.index.values))
test['release_date'].loc[test['release_date'].isin(unmatched)]=-1
mmo7=pd.concat([pd.DataFrame(mmo7.index.values),pd.DataFrame(mmo7.release_date.values)],axis=1)
mmo71=list(mmo7.iloc[30:,0])
data['release_date'].loc[data['release_date'].isin(mmo71)]=-1
test['release_date'].loc[test['release_date'].isin(mmo71)]=-1

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
del mmo3
del mmo31
del mmo4
del mmo5
del mmo6
del mmo7
del mmo41
del mmo51
del mmo61
del mmo71
del mmo98

In [6]:
target=data['is_listened']
del data['is_listened']
test2=test.iloc[:,1:]
data.head()
test2.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,artist_id_1,userartcluster,release_date_1,userdatecluster,userplafamcluster,userlistypecluster,userplnamecluster,usercontypecluster,useruagecluster,release_date1
0,50,1478104371,683078,-1,1,-1,0,0,542,1,...,2076,32,20021008,1,0,5,1,7,0,20021008
1,2744,1479317140,876497,-1,1,-1,0,0,307,1,...,26,33,19851231,6,0,7,1,8,4,19851231
2,2744,1479546361,876497,-1,1,-1,0,0,307,1,...,26,10,19851231,6,0,8,1,8,1,19851231
3,2744,1478457729,876500,-1,1,-1,2,1,265,1,...,26,40,19851231,6,7,5,8,3,0,19851231
4,2744,1480448560,876504,-1,1,-1,2,1,356,1,...,26,40,19851231,6,8,0,3,5,3,19851231


In [13]:
#test2.columns.tolist()

In [7]:
data=pd.concat([data,test2],axis=0)
data=data.fillna(0)
data.shape

(7578752, 35)

In [8]:
dummie1 = pd.get_dummies(data['context_type'], prefix='context_type', prefix_sep='_')
dummie2 = pd.get_dummies(data['platform_name'], prefix='platform_name', prefix_sep='_')
dummie3 = pd.get_dummies(data['platform_family'], prefix='platform_family', prefix_sep='_')
dummie4 = pd.get_dummies(data['listen_type'], prefix='listen_type', prefix_sep='_')
dummie5 = pd.get_dummies(data['user_gender'], prefix='user_gender', prefix_sep='_')
dummie6 = pd.get_dummies(data['genre_id'], prefix='genre_id', prefix_sep='_')
dummie7 = pd.get_dummies(data['album_id'], prefix='album_id', prefix_sep='_')
dummie8 = pd.get_dummies(data['release_date'], prefix='album_id', prefix_sep='_')
dummie9 = pd.get_dummies(data['usergencluster'], prefix='usergencluster', prefix_sep='_')
dummie10 = pd.get_dummies(data['artist_id'], prefix='artist_id', prefix_sep='_')
dummie11 = pd.get_dummies(data['usermedcluster'], prefix='usermedcluster', prefix_sep='_')
dummie12= pd.get_dummies(data['useralbcluster'], prefix='useralbcluster', prefix_sep='_')
dummie13= pd.get_dummies(data['userartcluster'], prefix='userartcluster', prefix_sep='_')
dummie14= pd.get_dummies(data['userdatecluster'], prefix='userdatecluster', prefix_sep='_')
dummie15 = pd.get_dummies(data['userplafamcluster'], prefix='userplafamcluster', prefix_sep='_')
dummie16= pd.get_dummies(data['userlistypecluster'], prefix='userlistypecluster', prefix_sep='_')
dummie17= pd.get_dummies(data['userplnamecluster'], prefix='userplnamecluster', prefix_sep='_')
dummie18= pd.get_dummies(data['usercontypecluster'], prefix='usercontypecluster', prefix_sep='_')
dummie19= pd.get_dummies(data['useruagecluster'], prefix='useruagecluster', prefix_sep='_')

In [9]:
data=pd.concat([data,dummie1,dummie2,dummie3,dummie4,dummie5,dummie6,dummie7,dummie8,dummie9,dummie10,dummie11,dummie12,dummie13,dummie14,dummie15,dummie16,dummie17,dummie18,dummie19],axis=1)
data.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,usercontypecluster_5,usercontypecluster_6,usercontypecluster_7,usercontypecluster_8,usercontypecluster_9,useruagecluster_0,useruagecluster_1,useruagecluster_2,useruagecluster_3,useruagecluster_4
0,25471,1480597215,222606,-1,12,-1,1,0,223,0,...,1,0,0,0,0,0,0,0,1,0
1,-1,1480544735,250467,-1,0,-1,2,1,171,0,...,0,0,1,0,0,1,0,0,0,0
2,-1,1479563953,305197,-1,1,-1,2,1,149,1,...,0,0,0,0,0,0,0,0,1,0
3,7,1480152098,900502,-1,0,-1,0,0,240,0,...,0,0,0,0,0,1,0,0,0,0
4,7,1478368974,542335,-1,0,-1,0,0,150,0,...,0,0,0,0,0,0,1,0,0,0


In [10]:
data.columns.get_loc("context_type_-1")

35

In [11]:
predictors = data.iloc[:,35:].as_matrix()# assign feature dataframe to predictors removing the target column
target1 = to_categorical(target)# assign target dataframe to y
n_cols = predictors.shape[1]
predictors.shape

(7578752, 648)

In [12]:
len(predictors)-len(test)

7558834

In [14]:
import xgboost as xgb

params = {"objective": "binary:logistic","eval_metric": "auc" ,"booster": "gbtree",
          "eta": 0.1, "colsample_bytree": 0.6, "min_child_weight": 0.7,
          "tree_method": "exact", "seed": "1992"}

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
dtrain = xgb.DMatrix(predictors[0:6000000], target[0:6000000])
dvalid = xgb.DMatrix(predictors[6000000:7558834], target[6000000:])
dtest = xgb.DMatrix(predictors[7558834:])

In [16]:
nrounds = 300
watchlist = [(dtrain, 'train')]
bst = xgb.train(params, dtrain, num_boost_round=nrounds, evals=watchlist, verbose_eval=20)

[0]	train-auc:0.718485
[20]	train-auc:0.775887
[40]	train-auc:0.77915
[60]	train-auc:0.782616
[80]	train-auc:0.785101
[100]	train-auc:0.786802
[120]	train-auc:0.788068
[140]	train-auc:0.789286
[160]	train-auc:0.790227
[180]	train-auc:0.791197
[200]	train-auc:0.791905
[220]	train-auc:0.792559
[240]	train-auc:0.793243
[260]	train-auc:0.794022
[280]	train-auc:0.794589


In [18]:
pred=bst.predict(dvalid)
from sklearn.metrics import roc_auc_score
roc_auc_score(target[6000000:], pred)

0.79157261289555536

In [19]:
test_preds = bst.predict(dtest)
submit = pd.DataFrame({'sample_id': test['sample_id'], 'is_listened': test_preds})
submit.to_csv("2605/m4_xgb_50_300.csv", index=False)

In [23]:
train_preds = bst.predict(dtrain)

In [27]:
sub = pd.DataFrame(train_preds, columns=['pred'])
sub1 = pd.DataFrame(pred, columns=['pred'])

In [28]:
sub=pd.concat([sub,sub1],axis=0)
len(sub)

7558834

In [30]:
#sub

In [31]:
sub.to_csv("train_pred/m4_xgb_50_300_2805.csv")

In [32]:
early_stopping_monitor = EarlyStopping(patience = 2)

# Model
model = Sequential()
model.add(Dense(96 , activation = 'relu' , input_shape = (n_cols,), kernel_initializer='random_normal',bias_initializer='zeros'))
model.add(Dropout(0.4))
model.add(Dense(96 , activation = 'relu', kernel_initializer='random_normal',bias_initializer='zeros'))
model.add(Dropout(0.4))
model.add(Dense(14 , activation = 'relu', kernel_initializer='random_normal',bias_initializer='zeros'))
model.add(Dropout(0.2))

model.add(Dense(2 , activation = 'softmax'))

model.compile(optimizer = 'adadelta' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [38]:
model.fit(predictors[:6000000] , target1[0:6000000] , validation_data = [predictors[6000000:7558834],target1[6000000:]] , epochs = 1 , callbacks = [early_stopping_monitor])

Train on 6000000 samples, validate on 1558834 samples
Epoch 1/1
6000000/6000000 [==============================] - 661s - loss: 0.5163 - acc: 0.7506 - val_loss: 0.5083 - val_acc: 0.7540


In [39]:
valid_preds = model.predict_proba(predictors[6000000:7558834], verbose=0)
valid_preds = valid_preds[:, 1]
roc = metrics.roc_auc_score(target1[6000000:,1], valid_preds)
roc

0.77994142292314395

In [40]:
preds = model.predict_proba(predictors[7558834:], verbose=0)[:, 1]
submission = pd.DataFrame(preds, columns=['is_listened'])
submission.to_csv('2605/m4_wd_2e_3l_2805_3.csv')
preds

array([ 0.84799325,  0.5139755 ,  0.70433617, ...,  0.77603471,
        0.71547002,  0.8107174 ], dtype=float32)

In [41]:
train_preds = model.predict_proba(predictors[:7558834], verbose=0)[:,1]
sub = pd.DataFrame(train_preds, columns=['is_listened'])
sub.shape

(7558834, 1)

In [42]:
sub.to_csv('train_pred/m4_wd_2e_3l_2805_3.csv')

In [ ]:
model_json = model.to_json()
with open("models/m4_wd_2e_3l_2805_3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/m4_wd_2e_3l_2805_3.h5")
print("Saved model to disk")